In [15]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline
import pprint as pp

import os
import numpy as np
import tensorflow as tf

import keras
from keras.preprocessing import image
from keras.applications.vgg19 import (
    VGG19, preprocess_input, decode_predictions)

In [16]:
# Load the VGG19 model
# https://keras.io/applications/#VGG19
model = VGG19(
    include_top=False, 
    weights='imagenet')
# model_original = VGG19(
#     include_top=True, 
#     weights='imagenet')

In [17]:
pp.pprint(model.layers)
pp.pprint(model.output)

<tf.Tensor 'block5_pool_1/MaxPool:0' shape=(?, ?, ?, 512) dtype=float32>


In [18]:
# Keras
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D
from keras.datasets import mnist

In [19]:
# add a global spatial average pooling layer
x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)


In [20]:
# this is the model we will train
model = Model(inputs=model.input, outputs=predictions)

In [38]:
# load image data
import glob
filtered_images_path= glob.glob('./imgs_with_filter/imgs_with_filter/*')
unfiltered_imgs_path=glob.glob('./unfiltered_1083/*')
images_path=filtered_images_path+unfiltered_imgs_path
images_list=[]
image_size = (299, 299)
for img_path in images_path:
    image_stored=image.load_img(img_path,target_size=image_size)
    
    # Preprocess image for model prediction
# This step handles scaling and normalization for Xception
    x = image.img_to_array(image_stored)
#     x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    images_list.append(x)
    
# X=images_list[0]

In [39]:
X=np.array(images_list)

In [40]:
filtered_labels=np.full((len(filtered_images_path),2),(True,False),np.bool)
unfiltered_labels=np.full((len(unfiltered_imgs_path),2),(False,True),np.bool)
y=np.concatenate([filtered_labels,unfiltered_labels])
type(y)

numpy.ndarray

In [41]:
#splitting data into test and train
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(images_list,y,test_size=0.33,shuffle=True)

In [31]:
from numpy import random
def generator(features, labels, batch_size):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, 299, 299, 3))
 batch_labels = np.zeros((batch_size,2))
 while True:
   for i in range(batch_size):
     # choose random index in features
     index= random.choice(len(features),1)
     batch_features[i] = (features[index])
     batch_labels[i] = labels[index]
   yield batch_features, batch_labels

In [34]:
#compile the model
for layer in model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [33]:
# train the model on the new data for a few epochs
# model.fit_generator("")
model.fit(X_train,y_train,batch_size=10,epochs=1,shuffle=True)

ValueError: Input arrays should have the same number of samples as target arrays. Found 1000 input samples and 2190 target samples.

In [35]:
#Train the model
model.fit_generator(generator(X_train, y_train, 10), steps_per_epoch=200, epochs=1, validation_data=generator(X_test,y_test) validation_steps=10)

Epoch 1/1
1000
109/200 [===============>..............] - ETA: 18:04 - loss: 0.2577

KeyboardInterrupt: 

In [32]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)

ValueError: `steps_per_epoch=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps_per_epoch` or use the `keras.utils.Sequence` class.